# DS4DS Homework Exercise Sheet 03

In this exercise you implement your own numerical solver for the 2D heat equation, which describes the tempearture $T$ in a 2D domain over the time interval $t\in[t_0=0,t_e=0.2]$:
$$\frac{\partial T}{\partial t} = \lambda \Delta T,$$ 
where 
$\Delta T = \frac{\partial^2 T}{\partial s_1^2} + \frac{\partial^2 T}{\partial s_2^2}$ is the _Laplace operator_ and $\lambda>0$ is a material parameter.

The system can be modeled by discrete approximation of the Laplace operator using the central difference method
$$\frac{\partial^2 T_{i,j}}{\partial s_1^2} \approx \frac{T_{i+1,j} - 2T_{i,j} + T_{i-1,j}}{\Delta s^2}, \qquad 
\frac{\partial^2 T_{i,j}}{\partial s_2^2} \approx \frac{T_{i,j+1} - 2T_{i,j} + T_{i,j-1}}{\Delta s^2}.$$

As a setup, consider a rectangular domain $\Omega = (0,1)^2$ with boundary 
$$\Gamma = [0,1] \times 0 ~\cup~ [0,1] \times 1 ~\cup~ 0 \times [0,1] ~\cup~ 1 \times [0,1]. $$ 
(i.e., bottom $\cup$ top $\cup$ left $\cup$ right.)

For discretization, use an equidistant grid with $n+2$ points in each direction (i.e., we have an $n+2 \times n+2$ grid in space). This means that $\Delta s = 1 / (n + 1)$. Of these $n+2$ grid points per dimension, the first and last are boundary nodes of Dirichlet type, meaning that they have a given value $T(s,t)= T_b(t)$ for $s\in\Gamma$. Consequently, the solution for the heat equation only has to be calculated for the interior of the domain. The solution we compute only concerns the $n^2$ interior grid nodes.

As this is more appropriate for numerical simulations and for later use of the resulting data, we will _vectorize_ (or _flatten_) the numerical solution into a vector $\hat{T}_i \in \mathbb{R}^{n^2}$ at every point $i$ in time. The indexing only addresses these interior points! This is visualized in the following sketch:

![](MatrixScetch.png)

### Task 1: Create the heat equation matrix - (4 points)

First, consider homogeneous boundary conditions, i.e., $T(s,t)=0$ for $s\in\Gamma$. This means that we can discretize the system of interior points by a matrix-vector multiplication $\Delta T \approx A \hat{T}$ without including any boundary conditions.
The resulting estimated derivative is formed as
$$\frac{\partial \hat{T}}{\partial t} = \lambda A \hat{T},$$ 
where $\hat{T} \in \mathbb{R}^{N} = \mathbb{R}^{n^2}$, $A \in \mathbb{R}^{N \times N} = \mathbb{R}^{n^2 \times n^2}$.



Write a function to automatically compute the matrix $A$ for arbitrary grid sizes $n$ (per dimension).

Since the numerical solution (spatially discretized) $\hat{T}$ will have the form of a vector (and not a matrix), the grid nodes should be numbered according to the following convention (see also the Figure above):

* first row (i.e., $s_2 = \Delta s$) goes from $1$ to $n$,
* second row (i.e, $s_2 = 2\Delta s$) goes from $n + 1$ to $2n$,
* ...

This results in the form

$$
    \hat{T} = \begin{bmatrix} \hat{T}_0 & \hat{T}_1 & \dots & \hat{T}_{n-1} & \hat{T}_{n} & \hat{T}_{n+1} & \dots & \hat{T}_{n^2 - 1} & \hat{T}_{n^2} \end{bmatrix}^\mathrm{T}
$$



**Hint**: It is helpful to explicitly draw an example for, e.g., $n=3$ or $n=4$ on paper and compute the matrix by hand. What patterns do you see? How can you translate those patterns into rules for the matrix?

In [ ]:
function create_heat_equation_matrix(n)
    
    ds = NaN  # to be filled
    factor = NaN  # to be filled
    
    #--- YOUR CODE STARTS HERE ---#
    
    #--- YOUR CODE ENDS HERE ---#

    N = n * n
    A = zeros(N, N)
    for i = 1:N  # iterate through the rows of the A matrix
        #--- YOUR CODE STARTS HERE ---#
        
        #--- YOUR CODE ENDS HERE ---#
    end
    return A
end;


If you are unable to properly solve this task. Use your imperfect version of the `create_heat_equation_matrix` for the following tasks. If everything except for the matrix function is implemented correctly, you will still get points for the subsequent tasks. 

That is, built solutions that would work if the function `create_heat_equation_matrix` is replaced with the correct version.

In [ ]:
# Public Test
@assert size(create_heat_equation_matrix(9)) == (81, 81)
@assert size(create_heat_equation_matrix(49)) == (2401, 2401)

In [ ]:
# please leave this cell as it is


In [ ]:
# Public Test
Test = [[-36. 9. 9. 0.];
        [9. -36. 0. 9.];
        [9. 0. -36. 9.];
        [0. 9. 9. -36.]]
@assert Test == create_heat_equation_matrix(2)

In [ ]:
# please leave this cell as it is


In [ ]:
# please leave this cell as it is


### Task 2: Utilizing the explicit euler for time stepping - (5 points)
Simulate the system (using the _Explicit Euler_ scheme for time stepping) with 

* $n=49$ (i.e., $\Delta s = 0.02$),
* $\Delta t=5 \cdot 10^{-5}$,
* $t_0 = 0$, 
* $t_e = 0.2$,
* initial state $T(s,0) = 1$ for $s\in\Omega$,
* $\lambda = 1$ ($\lambda$ should still be a parameter that can be changed).

Include a condition that identifies the time ```t_stat``` when a stationary state is almost reached ( $\| \hat{T}_i - \hat{T}_{i-1} \|_2 \leq 0.01$ ).
The solution function should return both the temperatures from $t_0$ until $t_e$ and the time satisfying the stationarity condition given above, i.e., ``` T, t_stat = heat_equation(n, λ, t_e, delta_t) ```.

**Hint 1**: You may use the `norm()` function from the `LinearAlgebra` package to compute $\|\cdot\|_2$.

**Hint 2**: The following two pictures visualize how the solution should look at t=0.05 and at t=0.1

t = 0.05                   |  t = 0.1
:-------------------------:|:-------------------------:
![](005.png)               | ![](01.png)


In [ ]:
using LinearAlgebra


function heat_equation(n, λ, t_e, delta_t)

    A = create_heat_equation_matrix(n)
 
    # number of iterations
    n_iter = Int(t_e / delta_t)
    
    # flag whether final state was found
    # when no termination occurs, return the -1
    t_stat = -1
    
    # create a matrix where T[:, 1] is the initial state, 
    # T[:, 2] is the state after the first iteration,
    # T[:, 3] is the state after the secound iteration, ... 
    
    # initial state T_0
    T_0 = vec(ones(n, n))
    
    # Store all the states for plotting
    T = Matrix{Float64}(undef, length(T_0), n_iter + 1) 
    T[:, 1] = T_0
    
    #--- YOUR CODE STARTS HERE ---#
    
    #--- YOUR CODE ENDS HERE ---#
    
    return T, t_stat 
end


In [ ]:
# Public Test
n = 49
λ = 1
t_e = 0.2
delta_t = 5 * 1e-5

T, t_stat = heat_equation(n, λ, t_e, delta_t)
@assert length(T[:,1]) == 2401
@assert length(T[:,2]) == 2401
@assert size(T,2) == 4001
@assert isa(t_stat, Number)
@assert isapprox(t_stat, 0.07025, atol=1e-10, rtol=1e-10)

In [ ]:
# please leave this cell as it is


In [ ]:
# please leave this cell as it is


In [ ]:
# please leave this cell as it is


### Task 3: Eigenvalue analysis - (2 points)

Examination of the eigenvalues of the matrix $A$ with the parameters of task 2. What can you say about the stability of the system? To this end, identify the eigenvalue ``` lambda_max ``` with the largest real part.

The system is:
- Stable                 -> stability = 1
- Asymptotically stable  -> stability = 2
- Unstable               -> stability = 3

In [ ]:
lambda_max = NaN
stability = NaN

n = 49

#--- YOUR CODE STARTS HERE ---#

#--- YOUR CODE ENDS HERE ---#

In [ ]:
# Public test
@assert isa(lambda_max, Number)
@assert isa(stability, Number)
@assert stability <= 3 && stability >= 1

In [ ]:
# please leave this cell as it is


In [ ]:
# please leave this cell as it is


### Task 4: Analysis of various material constants - (1 point)

To study the influence of the material constant $\lambda$, calculate the time until the system reaches the final state (using the same setup as in Task 2) for

* $\lambda = 0.5$
* $\lambda = 1.5$

Store the results in ```t_stat2``` and ```t_stat3```.

In [ ]:
t_stat2 = NaN # to be filled for lambda = 0.5
t_stat3 = NaN # to be filled for lambda = 1.5

n = 49
t_e = 0.2
delta_t = 5 * 1e-5

#--- YOUR CODE STARTS HERE ---#

#--- YOUR CODE ENDS HERE ---#

In [ ]:
# Public test
@assert isa(t_stat2, Number)
@assert isa(t_stat3, Number)

In [ ]:
# please leave this cell as it is


In [ ]:
# please leave this cell as it is
